In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as ex
from sklearn.preprocessing import normalize
from imblearn.over_sampling import SMOTE
from sklearn.model_selection import cross_val_score
import xgboost as xgb
from sklearn.model_selection import train_test_split
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import cross_val_predict
from sklearn.metrics import confusion_matrix
from sklearn.metrics import accuracy_score
from sklearn.metrics import f1_score
from sklearn.metrics import recall_score
# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
data = pd.read_csv('/kaggle/input/credit-card-customers/BankChurners.csv')
data.head()

In [ ]:
data.info()

In [ ]:
data.drop(['Naive_Bayes_Classifier_Attrition_Flag_Card_Category_Contacts_Count_12_mon_Dependent_count_Education_Level_Months_Inactive_12_mon_1'], inplace=True, axis=1)
data.drop(['Naive_Bayes_Classifier_Attrition_Flag_Card_Category_Contacts_Count_12_mon_Dependent_count_Education_Level_Months_Inactive_12_mon_2'], inplace=True, axis=1)

In [ ]:
data.head()

In [ ]:
data.describe()

In [ ]:
data = data.replace({'Less than $40K':20000, '$40K - $60K':40000, 
                     '$80K - $120K':80000, '$60K - $80K':60000, '$120K +':250000})


In [ ]:
data['Income_Category'].replace({'Unknown':20000}, inplace=True)

In [ ]:
data = data.replace({'Existing Customer':0, 'Attrited Customer':1})
data.head()


In [ ]:
data = data.replace({'F':0, 'M':1})
data.head()

In [ ]:
fig = ex.pie(data, names='Attrition_Flag', title = 'Chunc clients vs not chunc clients'  )
fig.show()

In [ ]:
plt.figure(figsize=(20,20))
sns.heatmap(data.corr(), annot=True)

In [ ]:
data = pd.get_dummies(data, columns = ['Card_Category', 'Education_Level', 'Marital_Status'])


In [ ]:
data.head()

In [ ]:
data=data.drop(columns=['CLIENTNUM'], axis=1)

In [ ]:
oversample = SMOTE()
X, y = oversample.fit_resample(data[data.columns[1:]], data[data.columns[0]])
upsample_data = X.assign(Churn=y)

In [ ]:
corr2 = upsample_data.drop(columns=upsample_data.columns[16:-1])
plt.figure(figsize=(20,20))
sns.heatmap(corr2.corr(), annot=True)

In [ ]:
upsample_data.head()

In [ ]:
X = upsample_data.drop(columns=['Churn'],axis=1)
y = upsample_data['Churn']

In [ ]:
X_train,X_test,y_train,y_test = train_test_split(X,y,random_state=50)
model = xgb.XGBClassifier()

In [ ]:
parameters = [{'learning_rate':[0.1,0.2,0.3,0.4],'max_depth':[3,4,5,6,7,8],'colsample_bytree':[0.5,0.6,0.7,0.8,0.9]}]
gscv = GridSearchCV(model, parameters, scoring = 'accuracy', n_jobs = -1, cv=10)
grid_search = gscv.fit(X,y)
grid_search.best_params_

In [ ]:
model = xgb.XGBClassifier(colsample_bytree=0.9, learning_rate=0.2, max_depth=7)
cvs = cross_val_score(model,X,y, scoring = 'accuracy', cv=10)
print('cross_val_score = ', cvs.mean())
y_pred = cross_val_predict(model,X,y,cv=10)
conf_mat = confusion_matrix(y_pred,y)
conf_mat


In [ ]:
model.fit(X_train,y_train)
pred = model.predict(X_test)
print('Recall= ', recall_score(y_test,pred))